# Task

* Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

Повторим шаги из заданий 1 и 2, используя библиотеку nltk.

* Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

# Additional links

Spacy - https://spacy.io/api/vocab

Emoji list - https://apps.timwhitlock.info/emoji/tables/unicode#block-4-enclosed-characters

Rem emoji from txt - https://www.educative.io/answers/how-to-remove-emoji-from-the-text-in-python

# I Libraries and Functions

## 1 Import Libraries

In [4]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 47.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=86403d96fc637b0235d3bb3d5b700d1d4b2a61310d7a99e708af7078721737b5
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [5]:
!pip install clean-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.8 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171034 sha256=ffffd40733b671d5a8333c769f7a5f5aa98cf594a300a0478839253d83e32a8f
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [108]:
import spacy
import os
import shutil
from google.colab import drive, files
import pandas as pd
import numpy as np
import nltk
import re
from tqdm import tqdm
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction import text as for_stop_word
from sklearn.base import BaseEstimator, TransformerMixin
from cleantext import clean
import emoji
from collections import Counter
import pickle
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## 2 Additional Functions

In [7]:
def MDir(dir_name):
    """Make a new folder"""
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)

def Upload(dir_name):
    f = files.upload()
    for fl in list(f.keys()):
        shutil.move('/content/' + fl, dir_name + fl)


# sw = set(for_stop_word.ENGLISH_STOP_WORDS)
# morpher = MorphAnalyzer()

class CleanText(BaseEstimator, TransformerMixin):

    """Cleaning text from regular characters in column
    key: name of column
    You shoul to import:
        >>> pip install pymorphy2
        >>> pip install clean-text
        import re
        import pandas as pd
        from tqdm import tqdm
        from pymorphy2 import MorphAnalyzer
        from sklearn.feature_extraction import text as for_stop_word
        from sklearn.base import BaseEstimator, TransformerMixin
        from cleantext import clean"""

    def __init__(self, key):
        self.key = key
        self.sw = set(for_stop_word.ENGLISH_STOP_WORDS)
        self.morpher = MorphAnalyzer()


    def clean_text(self, text:str) -> str:

        """Clean text"""

        if not isinstance(text, str):
            text = str(text)

        text = text.lower()
        text = text.strip('\n').strip('\r').strip('\t')
        text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))
        text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
        text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
        text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
        text = re.sub('\n', ' ', text)
        text = clean(text, no_emoji=True)
        text = [self.morpher.parse(word)[0].normal_form for word in text.split() if word not in self.sw]
        text = " ".join(text)
        return text


    def get_feature_names_out(self, feature_names_out=None):
        if feature_names_out != None:
            return feature_names_out
        else:
            return [self.key]

    def fit(self, X, y=None):
        return self

    def transform(self, X) -> pd.DataFrame:
        tqdm.pandas()
        X[self.key] = X[self.key].progress_apply(lambda x: CleanText.clean_text(self, x))
        return X[self.key]


def save_model_pkl(path, model_name):
    with open(path, "wb") as mod:
        pickle.dump(model_name, mod)


def load_model_pkl(path):
    with open(path, "rb") as mod1:
        model_name = pickle.load(mod1)
    return model_name

In [8]:
path_to_work_dir = "/content/drive/MyDrive/NLP_HW3/"

In [9]:
drive.mount('/content/drive/')

Mounted at /content/drive/


# II About Data

## 3 Load Data

In [ ]:
# MDir(path_to_work_dir)

In [ ]:
# Upload(path_to_work_dir)

Saving test_tweets.csv to test_tweets.csv
Saving train_tweets.csv to train_tweets.csv


In [161]:
df_train = pd.read_csv(path_to_work_dir + "train_tweets.csv")
df_test = pd.read_csv(path_to_work_dir + "test_tweets.csv")

In [47]:
df_train.head(7)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...


In [48]:
df_test.head(7)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."
5,31968,choose to be :) #momtips
6,31969,something inside me dies ð¦ð¿â¨ eyes nes...


## 4 Review and preprocessing data

### 4.1 Review

In [49]:
df_train.iloc[3,2]

'#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  '

### 4.2 Preprocessing

In [50]:
df = df_train

In [51]:
df.drop('id', axis=1, inplace=True)

In [52]:
CLT = CleanText('tweet')

In [53]:
df['tweet'] = CLT.transform(df)

100%|██████████| 31962/31962 [00:14<00:00, 2167.50it/s]


In [54]:
df.head(5)

,label,tweet
0,0,user father dysfunctional selfish drags kids d...
1,0,user user thanks lyft credit can't use cause d...
2,0,bihday majesty
3,0,model love u u time ur
4,0,factsguide society motivation


In [55]:
df.shape

(31962, 2)

In [56]:
df.iloc[3,1]

'model love u u time ur'

### 4.3 Save clean data

In [57]:
df.to_csv(path_to_work_dir + "clean_tweet.csv", index=None)

In [61]:
df = pd.read_csv(path_to_work_dir + "clean_tweet.csv")
df.head(7)

,label,tweet
0,0,user father dysfunctional selfish drags kids d...
1,0,user user thanks lyft credit can't use cause d...
2,0,bihday majesty
3,0,model love u u time ur
4,0,factsguide society motivation
5,0,huge fan fare big talking leave chaos pay disp...
6,0,user camping tomorrow user user user user user...


In [63]:
CLT = CleanText('tweet')
df['tweet'] = CLT.transform(df)

100%|██████████| 31962/31962 [00:11<00:00, 2870.72it/s]


In [111]:
corpus = " ".join(df['tweet'])
corpus = corpus[:999999]

In [112]:
corpus[-25:]

'elief htt lawofattraction'

# III SpaCy

## 5 Use the SpaCy

In [89]:
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

In [101]:
doc = nlp(corpus)

In [102]:
ents_dict = {}
for entity in doc.ents:
    if entity.label_ in ents_dict:
        ents_dict[entity.label_] += 1
    else:
        ents_dict[entity.label_] = 1

ents_dict

{'ORG': 978,
 'DATE': 3747,
 'GPE': 1347,
 'PERSON': 1828,
 'NORP': 511,
 'TIME': 625,
 'LAW': 2,
 'CARDINAL': 94,
 'LOC': 56,
 'EVENT': 10,
 'FAC': 25,
 'ORDINAL': 21,
 'PRODUCT': 38,
 'MONEY': 3,
 'LANGUAGE': 9,
 'QUANTITY': 4,
 'WORK_OF_ART': 3}

In [103]:
token_dict = {}
for token in doc:
    if token.pos_ in token_dict:
        token_dict[token.pos_] += 1
    else:
        token_dict[token.pos_] = 1

token_dict

{'PROPN': 32731,
 'ADJ': 17514,
 'NOUN': 59631,
 'VERB': 22253,
 'AUX': 4109,
 'PART': 2635,
 'PRON': 2866,
 'PUNCT': 1662,
 'INTJ': 896,
 'ADP': 997,
 'ADV': 3698,
 'DET': 130,
 'SCONJ': 282,
 'X': 206,
 'CCONJ': 113,
 'NUM': 94,
 'SYM': 23}

In [104]:
save_model_pkl(path_to_work_dir + "Model.pkl", doc)

In [105]:
doc2 = load_model_pkl(path_to_work_dir + "Model.pkl")

# IV NLTK

In [113]:
len(corpus)

999999

In [116]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [122]:
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

In [124]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [162]:
sentence = " ".join(df_train['tweet'])
sentence = sentence[:999999]

In [163]:
sentence[:25]

' @user when a father is d'

In [165]:
chunk_corpus = []
for sent in nltk.sent_tokenize(sentence):
  for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
     if hasattr(chunk, 'label'):
        chunk_corpus.append((chunk.label(), ' '.join(c[0] for c in chunk)))

In [166]:
chunk_corpus[:35]

[('GPE', 'en-ger-land'),
 ('GPE', 'u.s.'),
 ('ORGANIZATION', 'nycÂ'),
 ('ORGANIZATION', 'travelÂ'),
 ('ORGANIZATION', 'atÂ'),
 ('GPE', 'russia-england'),
 ('GPE', 'ð\x9f\x98\x8a'),
 ('GPE', 'u.s.'),
 ('ORGANIZATION', 'whiteaddictÂ'),
 ('ORGANIZATION', 'inc.'),
 ('GPE', 'u.s.'),
 ('ORGANIZATION', 'leastâ\x80¦'),
 ('ORGANIZATION', 'perÃº'),
 ('ORGANIZATION', 'sÃ³ria'),
 ('ORGANIZATION', '____________________________________'),
 ('ORGANIZATION', 'firstÂ'),
 ('ORGANIZATION', 'moanaÂ'),
 ('ORGANIZATION', 'ludik.hugo'),
 ('GPE', 'isrâ\x80¦'),
 ('ORGANIZATION', 'littleÂ')]

In [174]:
chunk_dict = {}
for chunk in chunk_corpus:
    if chunk[0] in chunk_dict:
        chunk_dict[chunk[0]] += 1
    else:
        chunk_dict[chunk[0]] = 1

chunk_dict

{'GPE': 7, 'ORGANIZATION': 13}

In [171]:
# TOP-20 популярных личностей
person_tokens_list = [(ent[0], ent[1]) for ent in chunk_corpus if ent[0] == 'PERSON']
Counter(person_tokens_list).most_common(20)

[]

In [173]:
# TOP-20 популярных организаций
org_tokens_list = [(ent[0], ent[1]) for ent in chunk_corpus if ent[0] == 'ORGANIZATION']
Counter(org_tokens_list).most_common(20)

[(('ORGANIZATION', 'nycÂ'), 1),
 (('ORGANIZATION', 'travelÂ'), 1),
 (('ORGANIZATION', 'atÂ'), 1),
 (('ORGANIZATION', 'whiteaddictÂ'), 1),
 (('ORGANIZATION', 'inc.'), 1),
 (('ORGANIZATION', 'leastâ\x80¦'), 1),
 (('ORGANIZATION', 'perÃº'), 1),
 (('ORGANIZATION', 'sÃ³ria'), 1),
 (('ORGANIZATION', '____________________________________'), 1),
 (('ORGANIZATION', 'firstÂ'), 1),
 (('ORGANIZATION', 'moanaÂ'), 1),
 (('ORGANIZATION', 'ludik.hugo'), 1),
 (('ORGANIZATION', 'littleÂ'), 1)]

# V Conclusion

In my opinion the "SpaCy" were better then nltk because it took less time for get the results and code. And nltk finded only two criterion while "SpaCy" more 10 criterion in the same text.

Maybe I did something wrong while use nltk.